In [2]:
import os

os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/"

import numpy as np
from libs import myDehankelization
import seaborn as sns
import matplotlib.pyplot as plt
import matlab
import pandas as pd
from random import randint
from scipy.io import savemat
from scipy.signal import resample
import ecg_plot
import scipy.fftpack as fftpack
from scipy.stats import kurtosis
from scipy.linalg import norm

/home/renan/Área de Trabalho/research-project/py37/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
df = pd.read_pickle('2022-02-02_02_05_35.pkl')
with open(f'classification_data/6d2698_indexes.npy', 'rb') as f:
    indexes = np.load(f)    
df_new = df[df.index.isin(indexes)]

In [ ]:
df_new = df_new.sample(frac=1)

In [ ]:
for index, row in df_new.iloc[0:500].iterrows():
    
    # Criar logica para pular caso exista
    
    recording = row['data']
    leads, samples = recording.shape
    new_freq = 100
    time_rec = samples/250
    n_samples = int(time_rec*new_freq)
    recording = resample(recording, n_samples, axis=1)
    
    T = np.zeros((50, 51, leads))
    my_hankelization = myHankelization.initialize()
    for i in range(0, leads):
        vec = matlab.double(recording[i].tolist(), size=recording[i].shape)
        T[:, :, i] = my_hankelization.hankelization(vec)
    my_hankelization.terminate()

    T = matlab.double(T.tolist(), size=T.shape)
    R = matlab.double([6.0], size=(1, 1))
    multirank = matlab.double([40.0, 40.0, 1.0], size=(1, 3))
    my_BTD = BTD3.initialize()
    output = my_BTD.BTD(R, multirank, T)
    output = np.array(output)
    my_BTD.terminate()
    
    with open(f"tensor_data/6d2698_indexes/{row.name}.npy", 'wb') as f:
        np.save(f, output)

In [ ]:
def get_sources(filename):
    with open(f"tensor_data/6d2698_indexes/{filename}", 'rb') as f:
            output = np.load(f, allow_pickle=True)
    output = np.array(output)
    sources = np.empty((0, 100), int)
    contrib_sources = np.empty((0, 12), int)
    my_dehankelization = myDehankelization.initialize()
    for i in range(0, len(output)):
        a = np.array(output[i][0])
        b = np.array(output[i][1]).T
        c = np.array(output[i][2]).T
        contrib_sources = np.append(contrib_sources, c, axis=0)
        mixing_matrix = np.dot(a, b)
        m_hankel = matlab.double(mixing_matrix.tolist(), size=mixing_matrix.shape)
        source = my_dehankelization.dehankelization(m_hankel)
        sources = np.append(sources, np.array(source).T, axis=0)
    my_dehankelization.terminate()
    return sources, contrib_sources


def get_likely_atrial(sources, contrib_sources):
    x = np.linspace(0, 1, 100)
    potentials_aa = np.empty((0, 100), int)
    for i, source in enumerate(sources):
        fourier = np.fft.fft(source)
        fourier = fourier[:int(len(source)/2)]
        frequencies = np.fft.fftfreq(len(source), d=x[1]-x[0])[:int(len(source)/2)]
        frequencies = frequencies[:int(len(source)/2)]
        freq = frequencies[np.argmax(np.abs(fourier))]
        contrib_source = contrib_sources[i, :]
        contrib_source_V1 = contrib_source[1]
        power_contrib = (norm(contrib_source_V1*source)**2)/100
        if freq > 3 and freq < 9 and power_contrib > 0.0001:
            potentials_aa = np.append(potentials_aa, np.array([source]), axis=0)
    return potentials_aa


def get_atrial_source(potentials_aa):
    kurt_pot_aa = [*map(lambda p_aa: kurtosis(p_aa), potentials_aa)]
    atrial_source = potentials_aa[kurt_pot_aa.index(max(kurt_pot_aa))]
    return atrial_source
    
    
def get_df_atrial():
    files = os.listdir('tensor_data/6d2698_indexes')
    rows = list()
    for filename in files:
        sources, contrib_sources = get_sources(filename)
        # potentials_aa = get_likely_atrial(sources, contrib_sources)
        #if potentials_aa.size != 0:
        #    atrial_source = get_atrial_source(potentials_aa)
        label = df.loc[int(filename.split('.')[0])].diagnostic
        rows.append({
              'id': int(filename.split('.')[0]),
              'data': sources,
              'label': label
        })
    return pd.DataFrame(rows)

df_atrial = get_df_atrial()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

df_atrial = pd.read_pickle('df_atrial.pkl')
label_encoder = LabelEncoder()
